In [1]:
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [18]:
data = pd.read_csv("/Users/vn0w1vm/Downloads/train 2.csv")

In [3]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
data = data[['text','label']]

In [5]:
data.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [20]:
df = data.copy()

In [21]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [22]:
df.head()

,text,label
0,House Dem Aide We Didn t Even See Comey s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired October ...,1
3,Videos Civilians Killed In Single US Airstr...,1
4,Print An Iranian woman has been sentenced to ...,1


In [23]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vn0w1vm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
df = df.loc[~(df['text'].isna())]

In [34]:
df = df.reset_index()

In [35]:
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

tokenized_doc[:5]

# # de-tokenization 
# detokenized_doc = [] 
# for i in range(len(df)): 
#     t = ' '.join(tokenized_doc[i]) 
#     detokenized_doc.append(t) 

# df['text'] = detokenized_doc

0    [House, Dem, Aide, We, Didn, Even, See, Comey,...
1    [Ever, get, feeling, life, circles, roundabout...
2    [Why, Truth, Might, Get, You, Fired, October, ...
3    [Videos, Civilians, Killed, In, Single, US, Ai...
4    [Print, An, Iranian, woman, sentenced, six, ye...
Name: text, dtype: object

In [36]:
# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [37]:
df.head()

,index,text,label
0,0,House Dem Aide We Didn Even See Comey Letter U...,1
1,1,Ever get feeling life circles roundabout rathe...,0
2,2,Why Truth Might Get You Fired October The tens...,1
3,3,Videos Civilians Killed In Single US Airstrike...,1
4,4,Print An Iranian woman sentenced six years pri...,1


In [38]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.2, random_state = 12)

In [39]:
df_trn.shape, df_val.shape

((16608, 3), (4153, 3))

In [40]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn,
                                      valid_df = df_val,
                                      vocab=data_lm.train_ds.vocab,
                                      bs=32)

/opt/anaconda3/lib/python3.7/site-packages/fastai/data_block.py:537: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
14294, 13428, 5535, 10294, 15968...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [44]:
learn = language_model_learner(data_lm,  arch = AWD_LSTM, pretrained = True, drop_mult=0.7)

In [45]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.773633,5.494880,0.268455,5:05:09
